In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from matplotlib import pyplot as plt
import scvelo as scv
import anndata as ad
import seaborn as sns
from matplotlib import rcParams

In [ ]:
sc.settings.verbosity=3
sc.logging.print_header()
sc.set_figure_params(dpi=300, fontsize=10)
FIGSIZE = (4, 3)
rcParams["figure.figsize"] = FIGSIZE

In [ ]:
oviduct=sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/oviductepitheliumforconcat.h5ad')
oviduct.var_names_make_unique()
oviduct = oviduct[~oviduct.obs['Patient'].isnull()]

In [ ]:
endo  = sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/endometrialepithelialclusteringfinalforconcat.h5ad')
endo.var_names_make_unique()

In [ ]:
oviendo = ad.concat([oviduct, endo], axis=0, join='inner')
oviendo
oviendo.obs['Cell_Types']

In [ ]:
epithelialcelltypes_dict= {'Ciliated (Endometrial)':['Ciliated (endometrial)'],
                           'Ciliated LGR5+ (Endometrial)':['Ciliated LGR5+'],
                           'Pre-Ciliated (Endometrial)':['Pre-Ciliated'],
                            'Mature multiciliated (Fallopian Tube)':['Mature multiciliated'],
                           'OVGP1+ ciliated (Fallopian Tube)':['OVGP1+ ciliated'],
                           'Pre-ciliated (Fallopian Tube)': ['Pre-ciliated (Fallopian tube)'],
                            'Mature secretory/Primary cilia (Fallopian Tube)':['Mature secretory (Primary cilia)'],
                            'Mature secretory-2 (Fallopian Tube)':['Mature secretory-2'],
                           'SOX9+ LGR5- (Endometrial)':['SOX9+ LGR5-'],
                            'Proliferative SOX9+ (Endometrial)' :['Proliferative SOX9+'],
                            'Glandular (Endometrial)':['Glandular'],
                           'Glandular Secretory (Endometrial)':['Glandular Secretory'],
                            'Mature Secretory-1 (Fallopian Tube)':['Mature secretory-1'],
                           'Progenitor (Fallopian Tube)':['Progenitor'],
                           'SOX9+ LGR5+ (Endometrial)':['SOX9+ LGR5+'],
                           'Lumenal 1 (Endometrial)':['Lumenal 1'],
                           'Lumenal 2 (Endometrial)':['Lumenal 2','Lumenal 3']}

oviendo.obs['epithelialcelltypes_tissue'] = np.nan

for i in epithelialcelltypes_dict.keys():
    ind = pd.Series(oviendo.obs.epithelial_celltypes).isin(epithelialcelltypes_dict[i])
    oviendo.obs.loc[ind,'epithelialcelltypes_tissue'] = i

oviendo.obs['epithelialcelltypes_tissue']

In [ ]:
tissue = {'Endometrium':['GA','Wang'],
          'Fallopian Tube':['Dinh_2021','Hu_2020','Ulrich_2022']}

oviendo.obs['Tissue_type'] = np.nan

for i in tissue.keys():
    ind = pd.Series(oviendo.obs.Author).isin(tissue[i])
    oviendo.obs.loc[ind,'Tissue_type'] = i

In [ ]:
sc.pp.combat(oviendo, key='Tissue')
sc.tl.pca(oviendo, svd_solver='arpack', n_comps=150)
sc.pp.neighbors(oviendo, n_neighbors=200, n_pcs=25)
sc.tl.leiden(oviendo,resolution=1.7)

sc.tl.paga(oviendo)
sc.pl.paga(oviendo)
sc.tl.umap(oviendo, init_pos='paga')


sc.pl.umap(oviendo, color=['Cell_Types','Author','leiden'])
scv.pl.proportions(oviendo)
scv.pp.moments(oviendo)
scv.tl.velocity(oviendo, fit_offset2=True)
scv.tl.velocity_graph(oviendo)
scv.pl.velocity_embedding_stream(oviendo, basis='umap', color='Cell_Types',
                                 legend_fontsize='6', legend_loc='right margin')

In [ ]:
sc.pl.umap(oviendo, color=['Tissue_type'])
sc.pl.umap(oviendo, color=['epithelialcelltypes_tissue'])

In [ ]:
marker_genes= ['TPPP3','FOXJ1','PIFO','CFAP157','C9orf24','SPAG6','OVGP1',
               'CRISP3','PGR','CXCL2','NOTCH2','ATP2B4','C3','LGR5','BCAM',
               'MSLN','VPS4A','SMO','FGFR1','PTCH1','MYC','ARMC4','DNAI1','LRRC6','PAEP']
sc.tl.dendrogram(oviendo, groupby='epithelialcelltypes_tissue')

sc.pl.dotplot(oviendo, marker_genes, groupby='epithelialcelltypes_tissue', 
              dendrogram=True, save = 'oviendoepithelialcelltypemarkersdotplot.png')

In [ ]:
results_file = ('/home/j87832lw/mounting/oviduct/data/oviendoumap.h5ad')
oviendo.write(results_file)